-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Ingesting Data Lab

Read in CSV files containing products data.

##### Tasks
1. Read with infer schema
2. Read with user-defined schema
3. Read with schema as DDL formatted string
4. Write using Delta format

In [0]:
%run ../Includes/Classroom-Setup

Python interpreter will be restarted.
Python interpreter will be restarted.

Skipping install of existing datasets to "dbfs:/mnt/dbacademy-datasets/apache-spark-programming-with-databricks/v03"


Validating the locally installed datasets...(1 seconds)

Creating & using the schema "da_duc_vu_5839_asp"...(1 seconds)

Predefined tables in "da_duc_vu_5839_asp":
 -none-

Predefined paths variables:
 DA.paths.datasets: dbfs:/mnt/dbacademy-datasets/apache-spark-programming-with-databricks/v03
 DA.paths.user_db: dbfs:/mnt/dbacademy-users/duc.vu@walgreens.com/apache-spark-programming-with-databricks/database.db
 DA.paths.working_dir: dbfs:/mnt/dbacademy-users/duc.vu@walgreens.com/apache-spark-programming-with-databricks
 DA.paths.checkpoints: dbfs:/mnt/dbacademy-users/duc.vu@walgreens.com/apache-spark-programming-with-databricks/_checkpoints
 DA.paths.sales: dbfs:/mnt/dbacademy-datasets/apache-spark-programming-with-databricks/v03/ecommerce/sales/sales.delta
 DA.paths.users: dbfs:/mnt/dbacademy-datasets/apache-spark-programming-with-databricks/v03/ecommerce/users/users.delta
 DA.paths.events: dbfs:/mnt/dbacademy-datasets/apache-spark-programming-with-databricks/v03/ecommerce/events/events.delta
 DA.paths.products: dbfs:/mnt/dbacademy-datasets/apache-spark-programming-with-databricks/v03/products/products.delta

Setup completed in 2 seconds

### 1. Read with infer schema
- View the first CSV file using DBUtils method **`fs.head`** with the filepath provided in the variable **`single_product_cs_file_path`**
- Create **`products_df`** by reading from CSV files located in the filepath provided in the variable **`products_csv_path`**
  - Configure options to use first line as header and infer schema

In [0]:
single_product_csv_file_path = f"{DA.paths.datasets}/products/products.csv/part-00000-tid-1663954264736839188-daf30e86-5967-4173-b9ae-d1481d3506db-2367-1-c000.csv"
print(dbutils.fs.head(single_product_csv_file_path ))

products_csv_path = f"{DA.paths.datasets}/products/products.csv"
products_df = (spark
           .read
           .option("sep", "\t")
           .option("header", True)
           .option("inferSchema", True)
           .csv(products_csv_path)
          )

products_df.printSchema()

item_id,name,price
M_PREM_Q,Premium Queen Mattress,1795.0
M_STAN_F,Standard Full Mattress,945.0
M_PREM_F,Premium Full Mattress,1695.0

root
-- item_id,name,price: string (nullable = true)

In [0]:
# TODO
single_product_csv_file_path = f"{DA.paths.datasets}/products/products.csv/part-00000-tid-1663954264736839188-daf30e86-5967-4173-b9ae-d1481d3506db-2367-1-c000.csv"
print(FILL_IN)

products_csv_path = f"{DA.paths.datasets}/products/products.csv"
products_df = FILL_IN

products_df.printSchema()

**1.1: CHECK YOUR WORK**

In [0]:
assert(products_df.count() == 12)
print("All test pass")

All test pass

### 2. Read with user-defined schema
Define schema by creating a **`StructType`** with column names and data types

In [0]:
from pyspark.sql.types import DoubleType, StringType, StructType, StructField
user_defined_schema = StructType([StructField("item_id", StringType(), True),
                                  StructField("name", StringType(), True),
                                  StructField("price", DoubleType(), True)
                                 ])

products_df2 = (spark
                .read
                .option("header", True)
                .schema(user_defined_schema)
                .csv(products_csv_path)
               )

In [0]:
products_df2.first()

Out[20]: Row(item_id='M_STAN_Q', name='Standard Queen Mattress', price=1045.0)

**2.1: CHECK YOUR WORK**

In [0]:
assert(user_defined_schema.fieldNames() == ["item_id", "name", "price"])
print("All test pass")

All test pass

In [0]:
from pyspark.sql import Row

expected1 = Row(item_id="M_STAN_Q", name="Standard Queen Mattress", price=1045.0)
result1 = products_df2.first()

assert(expected1 == result1)
print("All test pass")

All test pass

### 3. Read with DDL formatted string

In [0]:
ddl_schema = "`item_id` STRING, `name` STRING, `price` DOUBLE"

products_df3 = (spark
                .read
                .option("header", True)
                .schema(ddl_schema)
                .csv(products_csv_path)
               )

In [0]:
# TODO
ddl_schema = FILL_IN

products_df3 = FILL_IN

**3.1: CHECK YOUR WORK**

In [0]:
assert(products_df3.count() == 12)
print("All test pass")

All test pass

### 4. Write to Delta
Write **`products_df`** to the filepath provided in the variable **`products_output_path`**

In [0]:
products_output_path = f"{DA.paths.working_dir}/delta/products"
(products_df
 .write
 .format("delta")
 .mode("overwrite")
 .save(products_output_path)
)

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-945104582656939> in <module> 
 1 products_output_path = f"{DA.paths.working_dir}/delta/products" 
 ----> 2 (products_df
 3 . write
 4 . format ( "delta" ) 
 5 . mode ( "overwrite" ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in save (self, path, format, mode, partitionBy, **options) 
 738 self . _jwrite . save ( ) 
 739 else : 
 --> 740 self . _jwrite . save ( path ) 
 741 
 742 @ since ( 1.4 ) 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : 
Found invalid character(s) among " ,;{}()\n\t=" in the column names of your
schema. 
Please enable column mapping by setting table property 'delta.columnMapping.mode' to 'name'.
For more details, refer to https://docs.microsoft.com/azure/databricks/delta/delta-column-mapping
Or you can use alias to rename it.

In [0]:
# TODO
products_output_path = f"{DA.paths.working_dir}/delta/products"
products_df.FILL_IN

**4.1: CHECK YOUR WORK**

In [0]:
verify_files = dbutils.fs.ls(products_output_path)
verify_delta_format = False
verify_num_data_files = 0
for f in verify_files:
    if f.name == "_delta_log/":
        verify_delta_format = True
    elif f.name.endswith(".parquet"):
        verify_num_data_files += 1

assert verify_delta_format, "Data not written in Delta format"
assert verify_num_data_files > 0, "No data written"
del verify_files, verify_delta_format, verify_num_data_files
print("All test pass")

### Clean up classroom

In [0]:
DA.cleanup()

Resetting the learning environment...
...dropping the schema "da_duc_vu_5839_asp"...(1 seconds)
...removing the working directory "dbfs:/mnt/dbacademy-users/duc.vu@walgreens.com/apache-spark-programming-with-databricks"...(0 seconds)


Validating the locally installed datasets...(0 seconds)

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>